In [1]:
import pandas as pd
# import the reduce function to use for merging multiple dataframes later
from functools import reduce

### USDA Food Deserts Data

Source: https://www.ers.usda.gov/data-products/food-access-research-atlas/download-the-data/

In [2]:
# census tracts data
food_access = pd.read_csv('raw_data/food_deserts.csv', dtype={'CensusTract': object})
food_access.head()

,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,01001020100,Alabama,Autauga,1,1912,693,0,0,0.000000,0,...,221,1622,217,14,0,14,45,44,26,112
1,01001020200,Alabama,Autauga,1,2170,743,0,181,0.083410,0,...,214,888,1217,5,0,5,55,75,87,202
2,01001020300,Alabama,Autauga,1,3373,1256,0,0,0.000000,0,...,439,2576,647,17,5,11,117,87,108,120
3,01001020400,Alabama,Autauga,1,4386,1722,0,0,0.000000,0,...,904,4086,193,18,4,11,74,85,19,82
4,01001020500,Alabama,Autauga,1,10766,4082,0,181,0.016812,0,...,1126,8666,1437,296,9,48,310,355,198,488


In [3]:
# slicing county FIPS codes from CensusTranct column
food_access['FIPS'] = food_access['CensusTract'].apply(lambda x: x[:5])

**Variables of interest:**

1. **lapop1**: Low access, population beyond 1 mile from a supermarket, number
2. **lalowi1**: Low access, low-income population beyond 1 mile from a supermarket, number
3. **laseniors1**: Low access, seniors age 65+ beyond 1 mile from a supermarket, number
4. **lawhite1**: Low access, White population beyond 1 mile from a supermarket, number
5. **lablack1**: Low access, Black or African American population beyond 1 mile from a supermarket, number
6. **lahunv1**: Vehicle access, housing units without and low access beyond 1 mile from a supermarket, number
7. **lapop10**: Low access, population beyond 10 miles from a supermarket, number
8. **lalowi10**: Low access, low-income population beyond 10 miles from a supermarket, number
9. **laseniors10**: Low access, seniors age 65+ beyond 10 miles from a supermarket, number
10. **lawhite10**: Low access, White population beyond 10 miles from a supermarket, number
11. **lablack10**: Low access, Black or African American population beyond 10 miles from a supermarket, number
12. **lahunv10**: Vehicle access, housing units without and low access beyond 10 miles from a supermarket, number

`['lapop1', 'lalowi1', 'laseniors1', 'lawhite1', 'lablack1', 'lahunv1', 'lapop10', 'lalowi10', 'laseniors10', 'lawhite10', 'lablack10', 'lahunv10']`

**Aggregating tract-level data per county**

In [4]:
food_access = food_access.groupby(['State', 'County', 'FIPS'])['lapop1', 'lalowi1', 'laseniors1', 'lawhite1', 
                                                               'lablack1', 'lahunv1', 'lapop10', 'lalowi10', 
                                                               'laseniors10','lawhite10', 'lablack10',
                                                               'lahunv10'].sum().reset_index()
# drop the 'State' column
food_access.drop(['State'], axis=1, inplace=True)
food_access.head()

,County,FIPS,lapop1,lalowi1,laseniors1,lawhite1,lablack1,lahunv1,lapop10,lalowi10,laseniors10,lawhite10,lablack10,lahunv10
0,Autauga,01001,36469.872896,12899.819468,4245.641526,28592.192360,6645.162299,677.672769,7027.788425,2895.995077,892.652772,4393.008162,2469.136382,167.556997
1,Baldwin,01003,131649.158743,44343.516539,21633.863020,114569.054528,10709.565455,1394.162766,2913.544414,1135.997370,511.573835,2419.672998,393.186999,43.197231
2,Barbour,01005,19197.262382,9721.108548,2560.043174,9924.136304,8196.688099,425.144927,5501.798819,2732.660958,735.238254,2487.933427,2552.966247,160.010708
3,Bibb,01007,18918.377818,8937.631914,2286.891978,14775.827537,3775.872255,224.388125,295.614426,164.903295,56.362019,279.639427,10.924842,0.850700
4,Blount,01009,52242.222279,20568.194560,7301.222109,48763.477094,552.119031,719.931444,1955.518792,896.301342,264.590520,1875.942953,20.544442,19.115140


### Food Insecurity Data

**Reference**:
Gundersen, C., A. Dewey, M. Kato, A. Crumbaugh & M. Strayer. Map the Meal Gap 2019: A Report on County and Congressional District Food Insecurity and County Food Cost in the United States in 2017. Feeding America, 2019.

In [5]:
food_insecurity = pd.read_csv('raw_data/food_insecurity.csv', encoding = "ISO-8859-1", dtype={'FIPS': object})
food_insecurity['FIPS'] = food_insecurity['FIPS'].apply(lambda x: x.zfill(5))
food_insecurity.tail()

,FIPS,State,"County, State",2017 Food Insecurity Rate,# of Food Insecure Persons in 2017,Low Threshold in state,Low Threshold Type,High Threshold in state,High Threshold Type,% FI ? Low Threshold,% FI Btwn Thresholds,% FI > High Threshold,2017 Child food insecurity rate,# of Food Insecure Children in 2017,% food insecure children in HH w/ HH incomes below 185 FPL in 2017,% food insecure children in HH w/ HH incomes above 185 FPL in 2017,2017 Cost Per Meal,2017 Weighted Annual Food Budget Shortfall
3137,56037,WY,"Sweetwater County, Wyoming",10.7%,"4,750",130%,SNAP,185%,Other Nutrition Program,43.0%,12.8%,44.2%,17.0%,"2,030",51%,49%,$3.14,"$2,542,000.00"
3138,56039,WY,"Teton County, Wyoming",9.7%,"2,220",130%,SNAP,185%,Other Nutrition Program,36.0%,16.7%,47.3%,11.7%,520,56%,44%,$4.20,"$1,592,000.00"
3139,56041,WY,"Uinta County, Wyoming",12.8%,"2,660",130%,SNAP,185%,Other Nutrition Program,56.2%,7.4%,36.5%,18.9%,"1,160",64%,36%,$2.95,"$1,340,000.00"
3140,56043,WY,"Washakie County, Wyoming",11.2%,920,130%,SNAP,185%,Other Nutrition Program,50.5%,17.2%,32.3%,17.4%,350,74%,27%,$3.16,"$497,000.00"
3141,56045,WY,"Weston County, Wyoming",13.1%,940,130%,SNAP,185%,Other Nutrition Program,41.7%,11.9%,46.4%,20.2%,300,61%,40%,$3.15,"$505,000.00"


In [6]:
# selecting columns of interest only
food_insecurity = food_insecurity[['FIPS', 'County, State',
                                   '2017 Food Insecurity Rate', '# of Food Insecure Persons in 2017',
                                   '2017 Cost Per Meal']].copy()
# renaming the columns
food_insecurity.rename(columns={'2017 Food Insecurity Rate': 'insecure_perc',
                                '# of Food Insecure Persons in 2017': 'insecure_num',
                                '2017 Cost Per Meal': 'meal_cost'}, inplace=True)
food_insecurity.head()

,FIPS,"County, State",insecure_perc,insecure_num,meal_cost
0,01001,"Autauga County, Alabama",13.2%,"7,270",$3.19
1,01003,"Baldwin County, Alabama",11.6%,"23,560",$3.57
2,01005,"Barbour County, Alabama",22.0%,"5,760",$3.09
3,01007,"Bibb County, Alabama",14.3%,"3,240",$3.05
4,01009,"Blount County, Alabama",10.7%,"6,140",$3.10


### Health Factors Data

**Reference**: Early Stage Prediction of US County Vulnerability to the COVID-19 Pandemic Mihir Mehta, Juxihong Julaiti, Paul Griffin, Soundar Kumara medRxiv 2020.04.06.20055285; doi: https://doi.org/10.1101/2020.04.06.20055285

In [7]:
health = pd.read_csv('raw_data/health_factors.txt', dtype={'FIPS': object})
# make sure FIPS has five digits
health['FIPS'] = health['FIPS'].apply(lambda x: x.zfill(5))
# drop unnecsessary columns
health.drop(['State', 'TOT_POP', 'County', 'TOT_MALE', 'TOT_FEMALE',
             'GEOID', 'Latitude', 'Longitude', 'CGPS'], axis=1, inplace=True)
health.head()

,FIPS,pop_den_permile,diab_perc,hyper_perc,old_perc,CRUDE_RATE,CRD_MR
0,01001,91.8,9.9,40.38,0.11,553.5,81.79
1,01003,114.6,8.5,38.15,0.14,598.0,54.28
2,01005,31.0,15.7,48.28,0.13,546.0,69.82
3,01007,36.8,13.3,44.87,0.11,566.0,84.49
4,01009,88.9,14.9,41.81,0.13,551.0,87.05


### Poverty Data

Source: https://www.census.gov/data/datasets/2018/demo/saipe/2018-state-and-county.html

In [8]:
poverty = pd.read_csv('raw_data/poverty.csv', header=1, dtype={'State FIPS Code': object, 'County FIPS Code': object})
# renaming columns
poverty.rename(columns={'State FIPS Code': 'state_fips', 'County FIPS Code': 'county_fips',
                        'Poverty Estimate, All Ages': 'poverty_num', 'Poverty Percent, All Ages': 'poverty_per'}, inplace=True)
poverty.head()

,state_fips,county_fips,Postal Code,Name,poverty_num,poverty_per
0,00,000,US,United States,"41,852,315",13.1
1,01,000,AL,Alabama,"801,758",16.8
2,01,001,AL,Autauga County,"7,587",13.8
3,01,003,AL,Baldwin County,"21,069",9.8
4,01,005,AL,Barbour County,"6,788",30.9


In [9]:
# new column for complete county FIPS codes
poverty['FIPS'] = poverty['state_fips']+poverty['county_fips']
# drop unnecessary columns
poverty.drop(['Name', 'state_fips', 'county_fips', 'Postal Code'], axis=1, inplace=True)
poverty.head()

,poverty_num,poverty_per,FIPS
0,"41,852,315",13.1,00000
1,"801,758",16.8,01000
2,"7,587",13.8,01001
3,"21,069",9.8,01003
4,"6,788",30.9,01005


### Race Data

**Source**: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-detail.html?#

**Notes**:
1. only year 2018 data is used (i.e. YEAR = 11)
2. all-age-groups, total population data is used (i.e. AGEGRP = 0)

In [10]:
race = pd.read_csv('raw_data/population_by_race.csv', encoding = "ISO-8859-1", dtype={'STATE': object, 'COUNTY': object})
# make sure state fips have 2 digits
race['STATE'] = race['STATE'].apply(lambda x: x.zfill(2))
# make sure county fips have 3 digits
race['COUNTY'] = race['COUNTY'].apply(lambda x: x.zfill(3))
# create a column containing full county FIPS
race['FIPS'] = race['STATE']+race['COUNTY']
race.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE,FIPS
0,50,01,001,Alabama,Autauga County,11,0,55601,26995,28606,...,680,108,102,36,33,19,17,22,12,01001
1,50,01,003,Alabama,Baldwin County,11,0,218022,105657,112365,...,4516,259,253,259,192,69,57,56,34,01003
2,50,01,005,Alabama,Barbour County,11,0,24881,13133,11748,...,396,58,49,62,31,1,0,11,8,01005
3,50,01,007,Alabama,Bibb County,11,0,22400,11921,10479,...,228,31,16,7,14,1,0,18,3,01007
4,50,01,009,Alabama,Blount County,11,0,57840,28500,29340,...,2482,79,59,58,65,16,17,32,23,01009


**Variables of interest**:

1. **TOT_POP**: total population
2. **WA_MALE**: white alone male population
3. **WA_FEMALE**: white alone female population
4. **BA_MALE**: black alone male population
5. **BA_FEMALE**: black alone female population

`['TOT_POP', 'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE']`

In [11]:
# selecting variables of interest only
race = race[['STNAME', 'CTYNAME', 'FIPS',
             'TOT_POP', 'WA_MALE', 'WA_FEMALE', 'BA_MALE', 'BA_FEMALE']].copy()
race.head()

,STNAME,CTYNAME,FIPS,TOT_POP,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE
0,Alabama,Autauga County,01001,55601,20859,21801,5153,5762
1,Alabama,Baldwin County,01003,218022,92471,97830,9400,10092
2,Alabama,Barbour County,01005,24881,6410,5799,6370,5672
3,Alabama,Bibb County,01007,22400,8762,8449,2954,1816
4,Alabama,Blount County,01009,57840,27307,28149,507,443


In [12]:
# creating columns for % of total white, black, and non-white populations
race['white_per'] = ((race['WA_MALE']+race['WA_FEMALE'])/race['TOT_POP'])*100
race['black_per'] = ((race['BA_MALE']+race['BA_FEMALE'])/race['TOT_POP'])*100
race['nonwhite_per'] = 100 - race['white_per']
race.head()

,STNAME,CTYNAME,FIPS,TOT_POP,WA_MALE,WA_FEMALE,BA_MALE,BA_FEMALE,white_per,black_per,nonwhite_per
0,Alabama,Autauga County,01001,55601,20859,21801,5153,5762,76.725239,19.630942,23.274761
1,Alabama,Baldwin County,01003,218022,92471,97830,9400,10092,87.285228,8.940382,12.714772
2,Alabama,Barbour County,01005,24881,6410,5799,6370,5672,49.069571,48.398376,50.930429
3,Alabama,Bibb County,01007,22400,8762,8449,2954,1816,76.834821,21.294643,23.165179
4,Alabama,Blount County,01009,57840,27307,28149,507,443,95.878285,1.642462,4.121715


### COVID-19 Data
**(as of 04/16/2020)**

**Source**: https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv

In [13]:
covid = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv',
                    dtype={'fips': object})
# capitalize fips column name
covid.rename(columns={'fips': 'FIPS'}, inplace=True)
# setting the date column to be of type datetime
covid['date'] = pd.to_datetime(covid['date'])
print(covid.shape)
covid.head()

(64707, 6)


,date,county,state,FIPS,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


### Adding New York County Fips

NY Times dataset is missing the FIPS code for New York County, which is '36061'.

In [14]:
# NYC FIPS = '36061'
covid.loc[covid['county'] == 'New York City', 'FIPS'] = '36061'

**Note**: there are still missing FIPS for some counties; I only fixed NYC because of its importance since it has the highest number of cases in the nation and it needed to be fixed. The other ones shouldn't affect analysis later on.

-------------
### Merging All Datasets By Counties FIPS Codes

First, create a dataframe for the most recent total numbers of cases and deaths for each county. According NY Times, each row of data reports cumulative counts. So we only need to take the counts as of the most recent date in the data.

In [15]:
# most recent date
most_recent = covid['date'].max()
most_recent

Timestamp('2020-04-16 00:00:00')

In [16]:
# taking only most recent number of cases and deaths for each county
county_totals = covid[covid['date'] == most_recent]

In [19]:
print('As of {}/{}/{}, {} out of 3,141 counties have reported COVID-19 cases.'.format(most_recent.month, most_recent.day, most_recent.year, county_totals.shape[0]))

As of 4/16/2020, 2736 out of 3,141 counties have reported COVID-19 cases.


--------------
Second, create a dataframe combining most recent COVID-19 total cases and deaths for each county along with all demographic, poverty, food insecurity, and health factors.

**Note**: no temporal factors in this dataset

In [20]:
# compile the list of dataframes to be merged
dfs = [food_access, food_insecurity, health, poverty, race, county_totals]

In [21]:
# merge all dataframes on FIPS column
data = reduce(lambda  left,right: pd.merge(left,right,on=['FIPS'], how='outer'), dfs)

In [22]:
data.shape

(3228, 41)

**Note**: Some rows in the combined dataframe belong to non-county geographies (e.g. states). Using the 'race' dataframe as a reference, rows with missing values will be dropped. The 'race' dataframe contain data about counties only, and it has no missing values at all.

In [23]:
# keeping rows with no missing values in the 'white_per' column
# any other column that's unique to the 'race' dataframe could be selected
data = data[data['white_per'].notna()]

In [24]:
# drop date column
data.drop(['date'], axis=1, inplace=True)

In [25]:
data.shape

(3143, 40)

Some columns will still have some missing values. That's OK.

In [26]:
data.head()

,County,FIPS,lapop1,lalowi1,laseniors1,lawhite1,lablack1,lahunv1,lapop10,lalowi10,...,WA_FEMALE,BA_MALE,BA_FEMALE,white_per,black_per,nonwhite_per,county,state,cases,deaths
0,Autauga,01001,36469.872896,12899.819468,4245.641526,28592.192360,6645.162299,677.672769,7027.788425,2895.995077,...,21801.0,5153.0,5762.0,76.725239,19.630942,23.274761,Autauga,Alabama,25.0,2.0
1,Baldwin,01003,131649.158743,44343.516539,21633.863020,114569.054528,10709.565455,1394.162766,2913.544414,1135.997370,...,97830.0,9400.0,10092.0,87.285228,8.940382,12.714772,Baldwin,Alabama,102.0,2.0
2,Barbour,01005,19197.262382,9721.108548,2560.043174,9924.136304,8196.688099,425.144927,5501.798819,2732.660958,...,5799.0,6370.0,5672.0,49.069571,48.398376,50.930429,Barbour,Alabama,14.0,0.0
3,Bibb,01007,18918.377818,8937.631914,2286.891978,14775.827537,3775.872255,224.388125,295.614426,164.903295,...,8449.0,2954.0,1816.0,76.834821,21.294643,23.165179,Bibb,Alabama,24.0,0.0
4,Blount,01009,52242.222279,20568.194560,7301.222109,48763.477094,552.119031,719.931444,1955.518792,896.301342,...,28149.0,507.0,443.0,95.878285,1.642462,4.121715,Blount,Alabama,18.0,0.0


In [28]:
NYT_total = covid[covid['date']==most_recent]['cases'].sum()
my_total = data['cases'].sum()
print('Total number of confirmed cases according to NY Times data is: ', f"{NYT_total:,}")
print('Total number of confirmed cases in the resulting combined dataset: ', f"{my_total:,}")

Total number of confirmed cases according to NY Times data is:  668,542
Total number of confirmed cases in the resulting combined dataset:  661,907.0


### Writing COVID-19, and Combined Dataframes to CSV Files

In [29]:
# exporting county data
data.to_csv('county_data.csv', index=False)
# exporting daily COVID-19 data
covid.to_csv('covid_daily.csv', index=False)